In [144]:
import os
import pandas as pd

### Environment set-up

In [145]:
path = r"/Users/jafar/Downloads/cis 490/"
os.chdir(path)
sourcefiles = os.listdir(path)
[print(file) for file in sourcefiles if file.endswith(".csv")]

16k_Movies.csv
ClassicDisco.csv


[None, None]

### Reading Movies data

In [146]:
Movies_data = pd.read_csv("16k_Movies.csv", sep = ",")
Movies_data.head()

,Unnamed: 0,Title,Release Date,Description,Rating,No of Persons Voted,Directed by,Written by,Duration,Genres
0,0,Dekalog (1988),"Mar 22, 1996",This masterwork by Krzysztof Kieślowski is one...,7.4,118,Krzysztof Kieslowski,"Krzysztof Kieslowski, Krzysztof Piesiewicz",9 h 32 m,Drama
1,1,Three Colors: Red,"Nov 23, 1994",Krzysztof Kieslowski closes his Three Colors t...,8.3,241,Krzysztof Kieslowski,"Krzysztof Kieslowski, Krzysztof Piesiewicz, Ag...",1 h 39 m,"Drama,Mystery,Romance"
2,2,The Conformist,"Oct 22, 1970","Set in Rome in the 1930s, this re-release of B...",7.3,106,Bernardo Bertolucci,"Alberto Moravia, Bernardo Bertolucci",1 h 47 m,Drama
3,3,Tokyo Story,"Mar 13, 1972",Yasujiro Ozu’s Tokyo Story follows an aging co...,8.1,147,Yasujirô Ozu,"Kôgo Noda, Yasujirô Ozu",2 h 16 m,Drama
4,4,The Leopard (re-release),"Aug 13, 2004","Set in Sicily in 1860, Luchino Visconti's spec...",7.8,85,Luchino Visconti,"Giuseppe Tomasi di Lampedusa, Suso Cecchi D'Am...",3 h 7 m,"Drama,History"


### Exploring Movies dataset

In [147]:
Movies_data.shape
num_rows = Movies_data.shape[0]
num_cols = Movies_data.shape[1]

In [148]:
print("This dataset has {} rows and {} columns".format(num_rows, num_cols))

This dataset has 16290 rows and 10 columns


In [149]:
Movies_data.columns

Index(['Unnamed: 0', 'Title', 'Release Date', 'Description', 'Rating',
       'No of Persons Voted', 'Directed by', 'Written by', 'Duration',
       'Genres'],
      dtype='object')

In [150]:
Movies_data.dtypes

Unnamed: 0               int64
Title                   object
Release Date            object
Description             object
Rating                 float64
No of Persons Voted     object
Directed by             object
Written by              object
Duration                object
Genres                  object
dtype: object

In [151]:
# Renaming "unnamed id col" into "index"
Movies_data.rename(columns={'Unnamed: 0': 'index'}, inplace=True)

### What was the highest rated movie per year? 

In [165]:
Movies_data['Release Date'] = pd.to_datetime(Movies_data['Release Date'])
Movies_data['Year'] = Movies_data['Release Date'].dt.year
highest_rated_per_year = Movies_data.loc[Movies_data.groupby('Year')['Rating'].idxmax()][['Year', 'Title', 'Rating']]
highest_rated_per_year.head()

,Year,Title,Rating
638,1970,Patton,8.4
1941,1971,Get Carter,8.5
5,1972,The Godfather,9.3
1513,1973,Theater of Blood,9.0
254,1974,The Godfather: Part II,9.1


In [153]:
# Movies_data.to_csv("16k_Movies.csv")

### Which director has released the most number of movies over the years?

In [155]:
movies_by_director = Movies_data['Directed by'].value_counts().reset_index()
movies_by_director.head()

,index,Directed by
0,Woody Allen,50
1,Ron Howard,40
2,Clint Eastwood,35
3,Ridley Scott,35
4,Steven Spielberg,35


Woody Allen has the most number of movie release per year

### Does a particular genre always have a high rating? If so, what genre?

In [156]:
Movies_data['Genres'] = Movies_data['Genres'].str.split(',')  
exploded_genres = Movies_data.explode('Genres') # Spliting genre individually by rows
exploded_genres.head()

,index,Title,Release Date,Description,Rating,No of Persons Voted,Directed by,Written by,Duration,Genres,Year
0,0,Dekalog (1988),1996-03-22,This masterwork by Krzysztof Kieślowski is one...,7.4,118,Krzysztof Kieslowski,"Krzysztof Kieslowski, Krzysztof Piesiewicz",9 h 32 m,Drama,1996
1,1,Three Colors: Red,1994-11-23,Krzysztof Kieslowski closes his Three Colors t...,8.3,241,Krzysztof Kieslowski,"Krzysztof Kieslowski, Krzysztof Piesiewicz, Ag...",1 h 39 m,Drama,1994
1,1,Three Colors: Red,1994-11-23,Krzysztof Kieslowski closes his Three Colors t...,8.3,241,Krzysztof Kieslowski,"Krzysztof Kieslowski, Krzysztof Piesiewicz, Ag...",1 h 39 m,Mystery,1994
1,1,Three Colors: Red,1994-11-23,Krzysztof Kieslowski closes his Three Colors t...,8.3,241,Krzysztof Kieslowski,"Krzysztof Kieslowski, Krzysztof Piesiewicz, Ag...",1 h 39 m,Romance,1994
2,2,The Conformist,1970-10-22,"Set in Rome in the 1930s, this re-release of B...",7.3,106,Bernardo Bertolucci,"Alberto Moravia, Bernardo Bertolucci",1 h 47 m,Drama,1970


In [157]:
average_rating = exploded_genres.groupby('Genres')['Rating'].mean().reset_index()
average_rating = average_rating.sort_values(by="Rating", ascending=False).reset_index(drop=True)
average_rating.head()

,Genres,Rating
0,Animation,6.894456
1,War,6.839227
2,Biography,6.838772
3,History,6.801683
4,Music,6.775452


The Animation genre has the highest rating  with an average of 6.894456

### What is the longest movie by duration for each year?

In [162]:
Movies_data['Duration'] = pd.to_timedelta(Movies_data['Duration'],errors='coerce')
longest_movie_per_year = Movies_data.loc[Movies_data.groupby('Year')['Duration'].idxmax()][['Year', 'Title', 'Duration']]
longest_movie_per_year.head()

,Year,Title,Duration
638,1970,Patton,0 days 02:52:00
9918,1971,Nicholas and Alexandra,0 days 03:03:00
5,1972,The Godfather,0 days 02:55:00
3051,1973,The Iceman Cometh,0 days 03:59:00
254,1974,The Godfather: Part II,0 days 03:20:00


### Which director consistently receives the highest ratings?

In [163]:
director_ratings = Movies_data.groupby('Directed by')['Rating'].mean().reset_index()
director_ratings_sorted = director_ratings.sort_values(by='Rating', ascending=False).reset_index(drop=True)
director_ratings_sorted.head()

,Directed by,Rating
0,Hannah Pearl Utt,9.9
1,Pieter Gaspersz,9.9
2,Alice Englert,9.9
3,Agustín Díaz Yanes,9.8
4,"Micki Dickoff, \n \n Tony Pagano",9.8


Hannah Pearl Utt, Pieter Gaspersz and Alice Englert

### Which decade had the highest average movie ratings?

In [164]:
Movies_data['Decade'] = (Movies_data['Year'] // 10) * 10
decade_avg_rating = Movies_data.groupby('Decade')['Rating'].mean().reset_index()
decade_avg_rating_sorted = decade_avg_rating.sort_values(by='Rating', ascending=False)
decade_avg_rating_sorted

,Decade,Rating
0,1970,7.084091
1,1980,7.052645
2,1990,6.964243
3,2000,6.647415
4,2010,6.544136
5,2020,6.267323


1970 had the highest average movie ratings